In [1]:
pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


Shayanfazeli MIT-BIH Data

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shayanfazeli/heartbeat")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\shree\.cache\kagglehub\datasets\shayanfazeli\heartbeat\versions\1


In [4]:
print(os.listdir(path))
print("Dataset path",path)

['mitbih_test.csv', 'mitbih_train.csv', 'ptbdb_abnormal.csv', 'ptbdb_normal.csv']
Dataset path C:\Users\shree\.cache\kagglehub\datasets\shayanfazeli\heartbeat\versions\1


In [10]:
train = pd.read_csv(os.path.join(path,'mitbih_train.csv'), header = None)
test = pd.read_csv(os.path.join(path,'mitbih_test.csv'), header = None)

train.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
test.shape

(21892, 188)

In [45]:
train.shape

(87554, 188)

In [46]:
print(21892+87554)

109446


In [48]:
#109446 total samples

In [12]:
train[187].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: 187, dtype: float64

In [13]:
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]

X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]



In [16]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter =2000)

model.fit(X_train,y_train)

LogisticRegression(max_iter=2000)

In [17]:
y_pred = model.predict(X_test)

In [22]:
from sklearn.metrics import accuracy_score

accus = accuracy_score(y_pred,y_test)

print(accus*100, 'percent')

91.33930202813814 percent


In [23]:
from sklearn.metrics import classification_report

print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         0.0       0.99      0.92      0.95     19382
         1.0       0.37      0.84      0.51       245
         2.0       0.33      0.65      0.43       727
         3.0       0.30      0.58      0.40        84
         4.0       0.88      0.97      0.92      1454

    accuracy                           0.91     21892
   macro avg       0.57      0.79      0.64     21892
weighted avg       0.95      0.91      0.93     21892



In [32]:
sco = model.score(X_test,y_test)
print(sco)

0.9133930202813814


Quick Testing on new values

Converting samples into sin waves and then resampling it into 187 points

In [27]:
from scipy.signal import resample

def resample_signal_fft(signal, targrt_length = 187):
    resampled_signal = resample(signal,target_length)
    return resampled_signal




In [28]:
signal_1 = np.sin(np.linspace(0,4*np.pi,100))
signal_2 = np.cos(np.linspace(0,4*np.pi,150))
signal_3 = np.sin(np.linspace(0,4*np.pi,80))

target_length = 187

resampled_signals = np.array([
    resample_signal_fft(signal_1,target_length),
    resample_signal_fft(signal_2,target_length),
    resample_signal_fft(signal_3,target_length)
])

print("Resampled signal shape", resampled_signals.shape)

Resampled signal shape (3, 187)


In [29]:
predictions = model.predict(resampled_signals)
print("Pred", predictions)

Pred [1. 4. 1.]


As per Data:
Classes: ['N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4]

N: Normal
S: Supraventricular premature beat
V: Premature ventricular contraction
F: Fusion of ventricular and normal beat
Q: Unclassifiable beat

In [ ]:
print(np.unique(y_train))


[0. 1. 2. 3. 4.]


Cross Validation : To test the potential of the model on new data 
Using StratifiedKFolds to ensure proportional distribution of classes

In [36]:
from sklearn.model_selection import StratifiedKFold,cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    model,
    X_train,
    y_train,
    cv=cv,
    scoring = 'accuracy'

)

print("CV Accuracies", cv_scores)

CV Accuracies [0.9138827  0.9138256  0.91525327 0.91348295 0.91216448]
Mean CV accuracy 91.37217996057043


In [38]:
print("Mean CV accuracy", cv_scores.mean()*100) 

Mean CV accuracy 91.37217996057043


In [39]:
Testing_acc = model.score(X_test,y_test)
print("Testing Accuracy is", Testing_acc*100)

Testing Accuracy is 91.33930202813814


10 Fold CV

In [40]:
cvn = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cvn_scores = cross_val_score(
    model,
    X_train,
    y_train,
    cv=cvn,
    scoring = 'accuracy'

)

print("CV Accuracies", cvn_scores)

CV Accuracies [0.91560073 0.91263134 0.91434445 0.91308817 0.91182182 0.9184466
 0.91239292 0.91444889 0.91330668 0.91136493]


In [41]:
print("Mean CV accuracy", cvn_scores.mean()*100) 

Mean CV accuracy 91.37446526020894
